In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import matplotlib
from math import pow, sqrt, ceil
from collections import OrderedDict

from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

In [3]:
from Analysis.annotations import Annotation
from Analysis.image import Image
from Analysis.expert import Expert, DatasetType, ProjectType

In [4]:
from utils import *

In [5]:
path = Path("Results/EIPH/FirstRound")
files = glob(str(path)+"/Participants/*.txt")
gt_path = str(path)+"/GT/ground truth.txt"

users_first_round = []
ground_truth = Expert("GT", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
ground_truth.add_file(gt_path)

for file_name in files:
    participant = "Participant_" + Path(file_name).stem.split("_")[-1]
    
    expert = Expert(participant, BBType.Detected, DatasetType.EIPH_Exact, ProjectType.Annotation)
    expert.add_file(file_name)

    users_first_round.append(expert)

In [6]:
ground_truth

GT:  Annos: 237 Seconds: 0.0 Type: EIPH_Exact Project: Annotation

In [7]:
for user in users_first_round:
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))

Participant_11:  Annos: 220 Seconds: 57.25 Type: EIPH_Exact Project: Annotation mIoU: 0.4726323367046443
Participant_12:  Annos: 232 Seconds: 68.55 Type: EIPH_Exact Project: Annotation mIoU: 0.4784520856740563
Participant_13:  Annos: 180 Seconds: 55.9 Type: EIPH_Exact Project: Annotation mIoU: 0.458651975765022
Participant_3:  Annos: 213 Seconds: 29.75 Type: EIPH_Exact Project: Annotation mIoU: 0.4010948335671599
Participant_4:  Annos: 236 Seconds: 99.55 Type: EIPH_Exact Project: Annotation mIoU: 0.5981276645470117
Participant_5:  Annos: 190 Seconds: 36.0 Type: EIPH_Exact Project: Annotation mIoU: 0.5147334479279967
Participant_6:  Annos: 199 Seconds: 43.75 Type: EIPH_Exact Project: Annotation mIoU: 0.5335422857216442
Participant_7:  Annos: 212 Seconds: 34.25 Type: EIPH_Exact Project: Annotation mIoU: 0.42720048144293327
Participant_8:  Annos: 197 Seconds: 29.8 Type: EIPH_Exact Project: Annotation mIoU: 0.3879706578366469
Participant_9:  Annos: 182 Seconds: 56.9 Type: EIPH_Exact Projec

In [8]:
deleted_path = str(path)+"/GT/deleted.txt"
changed_path = str(path)+"/GT/changed.txt"
nms_path = str(path)+"/GT/nms.txt"


deleted_eiph = Expert("deleted", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
deleted_eiph.add_file(deleted_path)

changed_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
changed_eiph.add_file(changed_path)

nms_eiph = Expert("changed", BBType.GroundTruth, DatasetType.EIPH_Exact, ProjectType.Annotation)
nms_eiph.add_file(nms_path)


metrics = []
for user in users_first_round:
    total_changed_cells = 0
    found_changed_cells = 0
    for metric in user.calc_metrics(changed_eiph):
        total_changed_cells += int(metric['total positives'])
        found_changed_cells += int(metric['total TP'])
        
    total_deleted_cells = 0
    found_deleted_cells = 0
    for metric in user.calc_metrics(deleted_eiph):
        total_deleted_cells += int(metric['total positives'])
        found_deleted_cells += int(metric['total TP'])
        
    total_nms_cells = 0
    found_nms_cells = 0
    for metric in user.calc_metrics(nms_eiph):
        total_nms_cells += int(metric['total positives'])
        found_nms_cells += int(metric['total TP'])
        
    
    metrics.append([user.expert, total_changed_cells, found_changed_cells, total_deleted_cells, found_deleted_cells, total_nms_cells, found_nms_cells])
pd.DataFrame(metrics, columns=['Name', 'Total Changed', "Found Changed", "Total Deleted", "Found Deleted", "Total NMS", "Found NMS"])

D:\ProgProjekte\Python\Object-Detection-Metrics\lib\Evaluator.py:126: RuntimeWarning: invalid value encountered in true_divide
  rec = acc_TP / npos


,Name,Total Changed,Found Changed,Total Deleted,Found Deleted,Total NMS,Found NMS
0,Participant_11,60,42,5,2,10,3
1,Participant_12,60,34,5,1,10,3
2,Participant_13,60,37,5,1,10,1
3,Participant_3,60,31,5,0,10,2
4,Participant_4,60,44,5,3,10,4
5,Participant_5,60,39,5,0,10,1
6,Participant_6,60,41,5,2,10,1
7,Participant_7,60,34,5,0,10,1
8,Participant_8,60,22,5,1,10,1
9,Participant_9,60,35,5,4,10,1
